In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# !cp "/content/drive/Shareddrives/Character Recognition/DataSet/new datasets/archive_2.zip" "archive_2.zip"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
!mkdir "eng_chars/"

In [ ]:
!unzip -q /content/archive_2.zip -d "eng_chars/"

In [ ]:
!pip install -q scikit-image --upgrade

     |████████████████████████████████| 29.2 MB 68.9 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
from skimage.io import imread,imsave
from skimage.registration import phase_cross_correlation
from skimage.transform import warp_polar, rotate,rescale
from skimage.util import img_as_float,img_as_ubyte
import numpy as np
from tensorflow.data import AUTOTUNE
import os
import pandas as pd
import cv2
import tensorflow as tf
from sklearn.model_selection import train_test_split
import scipy as sp
import imutils

In [ ]:
df=pd.read_csv("/content/eng_chars/english.csv")
train, test = train_test_split(df, test_size=0.2, random_state=0, stratify=df[['label']])
train, val = train_test_split(train, test_size=0.1, random_state=0, stratify=train[['label']])

In [ ]:
train.reset_index(drop=True,inplace=True)
test.reset_index(drop=True,inplace=True)
val.reset_index(drop=True,inplace=True)
len(train)+len(test)+len(val)==len(df)

True

In [ ]:
def get_feature_vector(x,downscale=5):
  try:
    base_path="/content/eng_chars"
    img_path=os.path.join(base_path,x)
    gray=imread(img_path,as_gray=True)
    w, h = np.array(gray.shape)[:2] / 2
    radius = np.sqrt(w ** 2 + h ** 2)
    polar=img_as_ubyte(warp_polar(gray, radius=radius,scaling='log'))
    polar=imutils.resize(polar,width=polar.shape[1]//downscale)
    th, threshold = cv2.threshold(polar, 50, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
    cnts, hiers = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)[-2:]
    #sort contours and fit ellipse to biggest contours assuming biggest is our required
    cnt=max(cnts,key=lambda p: len(p))
    ellipse = cv2.fitEllipse(cnt)
    mask=np.zeros(polar.shape,dtype='uint8')
    mask=cv2.ellipse(mask,ellipse,255,-1)
    masked = cv2.bitwise_and(threshold, threshold, mask=mask)
    data=np.argwhere(masked==255)
    mean=np.mean(data)
    cov = np.cov(data.T)
    inv_covmat = sp.linalg.inv(cov)
    fv=list()
    for i in data:
      x_minus_mu = i - mean
      left_term = np.dot(x_minus_mu, inv_covmat)
      mahal = np.dot(left_term, x_minus_mu.T)
      fv.append(mahal)
    return np.array(fv)
  except:
    print(x)
    return None

In [ ]:
train['feature_vector']=train['image'].apply(lambda x:get_feature_vector(x))
train.to_csv("train.csv")

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: Degrees of freedom <= 0 for slice


Img/img025-006.png
Img/img017-025.png


In [ ]:
test['feature_vector']=test['image'].apply(lambda x:get_feature_vector(x))
test.to_csv("test.csv")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
val['feature_vector']=val['image'].apply(lambda x:get_feature_vector(x))
val.to_csv("val.csv")

In [ ]:
train_final=train.dropna()
train_final.to_csv("train_final.csv")

In [ ]:
num_classes=df['label'].nunique()
print(num_classes)

62


In [ ]:
trainX=train_final['feature_vector'].values
padded_inputs=tf.keras.preprocessing.sequence.pad_sequences(trainX,padding='post')
padded_inputs=padded_inputs.astype('int')
trainX=np.expand_dims(padded_inputs,axis=-1)
print(trainX.shape)

(2453, 7530, 1)


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
label_encoder = LabelEncoder()
trainY=train_final['label'].values
print(trainY.shape)
integer_encoded = label_encoder.fit_transform(trainY)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded.shape)
trainY=onehot_encoded

(2453,)
(2453, 62)


In [ ]:
testX=test['feature_vector'].values
padded_inputs=tf.keras.preprocessing.sequence.pad_sequences(testX,padding='post')
padded_inputs=padded_inputs.astype('int')
testX=np.expand_dims(padded_inputs,axis=-1)
print(testX.shape)
label_encoder = LabelEncoder()
testY=test['label'].values
print(testY.shape)
integer_encoded = label_encoder.fit_transform(testY)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded.shape)
testY=onehot_encoded

(682, 7347, 1)
(682,)
(682, 62)


In [ ]:
valX=val['feature_vector'].values
padded_inputs=tf.keras.preprocessing.sequence.pad_sequences(valX,padding='post')
padded_inputs=padded_inputs.astype('int')
valX=np.expand_dims(padded_inputs,axis=-1)
print(valX.shape)
label_encoder = LabelEncoder()
valY=val['label'].values
print(valY.shape)
integer_encoded = label_encoder.fit_transform(valY)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded.shape)
valY=onehot_encoded

(273, 7410, 1)
(273,)
(273, 62)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,LSTM,Embedding
from tensorflow.keras.optimizers import Adam
from keras import Input
def get_model(num_classes):
  model = Sequential()
  model.add(Embedding(input_dim=1000, output_dim=512, mask_zero=True))
  #model.add(LSTM(512,return_sequences=True,input_shape=(None,1)))
  model.add(LSTM(100))
  model.add(Dropout(0.2))
  #model.add(Dense(128))
  model.add(Dense(num_classes, activation='softmax'))
  adam=Adam(learning_rate=0.001)
  model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
  return model

In [ ]:
def create_callbacks():
    cpk_path = f'./models/best_model.h5'
    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath=cpk_path,
        monitor= 'val_accuracy',
        mode='max',
        save_best_only=True,
        verbose=1,
    )
    reducelr = tf.keras.callbacks.ReduceLROnPlateau(
        monitor= 'val_accuracy',
        mode='max',
        factor=0.1,
        patience=3,
        verbose=1
    )
    earlystop = tf.keras.callbacks.EarlyStopping(
        monitor= 'val_accuracy',
        mode='max',
        patience=10,
        verbose=1
    )
    callbacks = [checkpoint, reducelr, earlystop]
    return callbacks

In [ ]:
tf.keras.backend.clear_session()
with tf.device('/device:GPU:0'):
  callbacks=create_callbacks()
  model=get_model(num_classes)
  history=model.fit(trainX,trainY,epochs=200,batch_size=16,callbacks=callbacks,validation_data=(valX,valY),verbose=1)

In [ ]:
import matplotlib.pyplot as plt
plt.style.use("ggplot")
plt.figure()
plt.plot(history.history["loss"], label="train_loss")
plt.plot(history.history["val_loss"], label="val_loss")
plt.plot(history.history["accuracy"], label="train_accuracy")
plt.plot(history.history["val_accuracy"], label="val_accuracy")
plt.plot(history.history["lr"], label="Learning rate")
plt.title("Training Loss")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend(loc="best")
plt.savefig("plots.png",dpi=300)

In [ ]:
results=model.evaluate(x=testX,y=testY)

22/22 [==============================] - 22s 937ms/step - loss: 4.0302 - accuracy: 0.0381


[4.03017520904541, 0.03812316805124283]